<a href="https://colab.research.google.com/github/prafullahas/Emotion_detection/blob/main/vocab_coach_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy streamlit requests matplotlib
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import requests
import sqlite3


In [ ]:
nlp = spacy.load("en_core_web_sm")
API_URL = "https://api.dictionaryapi.dev/api/v2/entries/en/"

In [ ]:
conn = sqlite3.connect('vocab.db')
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS known_words(word TEXT)")
c.execute("CREATE TABLE IF NOT EXISTS unknown_words(word TEXT, meaning TEXT)")
conn.commit()

In [ ]:
def get_meaning(word):
    res = requests.get(API_URL + word)
    if res.status_code == 200:
        data = res.json()
        return data[0]['meanings'][0]['definitions'][0]['definition']
    return "Meaning not found."

def process_text(text):
    doc = nlp(text)
    words = set([token.lemma_.lower() for token in doc if token.is_alpha])
    c.execute("SELECT word FROM known_words")
    known = {row[0] for row in c.fetchall()}
    unknown = words - known
    results = []
    for w in unknown:
        meaning = get_meaning(w)
        results.append((w, meaning))
        c.execute("INSERT OR IGNORE INTO unknown_words VALUES (?,?)", (w, meaning))
    conn.commit()
    return results


In [ ]:
text_sample = "This magnificent cathedral was constructed during the medieval era."
new_words = process_text(text_sample)
print("New Words Found:")
for w, m in new_words:
    print(w, ":", m)


New Words Found:
magnificent : Grand, elegant or splendid in appearance.
be : Meaning not found.
construct : Something constructed from parts.
during : To last, continue, endure.
cathedral : Relating to the throne or the see of a bishop.
this : Something being indicated that is here; one of these.
era : A time period of indeterminate length, generally more than one year.
the : With a comparative or with more and a verb phrase, establishes a correlation with one or more other such comparatives.
medieval : Someone living in the Middle Ages.


In [ ]:
c.execute("SELECT word, meaning FROM unknown_words ORDER BY RANDOM() LIMIT 5")
quiz = c.fetchall()
for w, m in quiz:
    print("Word:", w)
    print("Meaning:", m)


Word: be
Meaning: Meaning not found.
Word: this
Meaning: Something being indicated that is here; one of these.
Word: medieval
Meaning: Someone living in the Middle Ages.
Word: construct
Meaning: Something constructed from parts.
Word: era
Meaning: A time period of indeterminate length, generally more than one year.
